# Module 5.2: 50-Qubit Inference (Utility Scale)

**The Concept:** Running a "toy" model on a simulator is easy. Running a utility-scale quantum kernel on 50+ qubits is engineering.

Here, we deploy a **Quantum Neural Network** classifier on the **IBM Torino** processor (or similar Heron-class hardware).

**The Stack:**
1.  **Feature Map:** ZZFeatureMap (Encodes 50 data features into entanglement).
2.  **Ansatz:** EfficientSU2 (A hardware-efficient variational circuit).
3.  **Error Mitigation:** We use **ZNE (Zero Noise Extrapolation)** to get clean results despite the noise of 50 qubits.

## 🧠 Hardware Access
xperiment require an IBM Quantum API key.  
Get one here: [quantum.ibm.com](https://quantum.ibm.com)

In [ ]:
import numpy as np
import time
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.circuit.library import EfficientSU2, ZZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions

# --- CONFIGURATION ---
# REPLACE WITH YOUR API KEY
API_KEY = "YOUR_IBM_QUANTUM_API_KEY_HERE"
BACKEND_NAME = "ibm_torino"
NUM_QUBITS = 50

print(f"Targeting: {BACKEND_NAME} | Qubits: {NUM_QUBITS}")

In [ ]:
class LargeScaleQuantumAI:
    def __init__(self):
        print(f"Initializing {NUM_QUBITS}-Qubit AI Model...")

        # 1. Setup Service
        self.service = QiskitRuntimeService(
            channel="ibm_quantum_platform",
            token=API_KEY
        )
        try:
            self.backend = self.service.backend(BACKEND_NAME)
        except Exception:
            # Fallback for users without access to Torino
            print(f"Warning: {BACKEND_NAME} not found. Using least busy available backend.")
            self.backend = self.service.least_busy(simulator=False, operational=True)
            print(f"Using: {self.backend.name}")

        # 2. Define the "Neural Network" Structure
        # A. Feature Map: Encodes data into the 50 qubits
        self.feature_map = ZZFeatureMap(feature_dimension=NUM_QUBITS, reps=1, entanglement='linear')

        # B. The Brain (Ansatz): The trainable layers
        # EfficientSU2 is efficient for hardware (less gates, more entanglement)
        self.ansatz = EfficientSU2(num_qubits=NUM_QUBITS, reps=1, entanglement='linear')

        # Combine them: Data Input -> Quantum Processing
        self.circuit = self.feature_map.compose(self.ansatz)

        # 3. Transpile circuit for the specific chip hardware
        print("Compiling circuit for physical hardware (ISA)...")
        pm = generate_preset_pass_manager(backend=self.backend, optimization_level=3)
        self.isa_circuit = pm.run(self.circuit)

        # 4. Define Observable (What we measure)
        # Measuring Z on the first qubit tells us the classification (1 or -1)
        observable = SparsePauliOp(["I" * (NUM_QUBITS - 1) + "Z"])
        self.isa_observable = observable.apply_layout(self.isa_circuit.layout)

    def predict(self, input_data, weights):
        """
        Runs a forward pass on the Quantum Computer.
        """
        print(f"--- STARTING INFERENCE ON {self.backend.name} ---")

        # 1. Configure Error Mitigation (ZNE)
        # This allows us to see the 'true' quantum state through the noise
        options = EstimatorOptions()
        options.resilience_level = 2
        options.resilience.zne_mitigation = True
        options.default_shots = 2048 

        estimator = Estimator(mode=self.backend, options=options)

        # 2. Combine Data and Weights into one parameter list
        # The circuit expects [Input_Data_Params, Weight_Params]
        full_params = np.concatenate([input_data, weights])

        print(f"Input Vector Size: {len(input_data)}")
        print(f"Weight Vector Size: {len(weights)}")
        print("Submitting job...")

        start_t = time.time()

        # 3. Run Job
        job = estimator.run([(self.isa_circuit, [self.isa_observable], [full_params])])
        print(f"Job ID: {job.job_id()}")
        print("Status: In Queue (This is real hardware, please wait...)")

        result = job.result()

        # 4. Extract Result
        # The result represents the neural activation (-1.0 to 1.0)
        expectation_value = result[0].data.evs[0]
        std_error = result[0].data.stds[0]

        print(f"Done in {time.time() - start_t:.2f}s")
        return expectation_value, std_error

In [ ]:
# --- EXECUTION ---

# 1. Instantiate the AI
ai = LargeScaleQuantumAI()

# 2. Generate Dummy Data
# Input: 50 random numbers (representing pixels or financial features)
input_vector = np.random.uniform(0, np.pi, NUM_QUBITS)

# Weights: Random weights for the ansatz
num_weights = ai.ansatz.num_parameters
weight_vector = np.random.uniform(0, 2*np.pi, num_weights)

# 3. Run Inference
prediction, error = ai.predict(input_vector, weight_vector)

# 4. Display Results
print("\n" + "="*40)
print("      50-QUBIT AI PREDICTION")
print("="*40)
print(f"Raw Output (Expectation): {prediction:.5f}")
print(f"Uncertainty (+/-):        {error:.5f}")
print("-" * 40)

# Interpret the result as a Binary Classification
if prediction > 0:
    print("AI Decision: CLASS A (Probability > 50%)")
else:
    print("AI Decision: CLASS B (Probability > 50%)")
print("="*40)